In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import statsmodels.api as sm
import statsmodels.formula.api as smf

from itertools import product
from ukbb_recessive.regression.regressions import run_regressions, save_table_for_paper, get_formula, get_target_family



# Load regressions datasets

First, we load datasets for the regression analysis saved on the previous step using script `0_create_dataset.ipynb`. 

In [2]:
output_path = "../../../../data/tables"
datasets_path = '.../450k/datasets'

datasets_dict = {
    'Cassa': pd.read_csv(f"{datasets_path}/Cassa.csv", sep='\t'),
    'pLI': pd.read_csv(f"{datasets_path}/PLI.csv", sep='\t'),
    'Weghorn': pd.read_csv(f"{datasets_path}/Weghorn-drift.csv", sep='\t'),
    'Roulette': pd.read_csv(f"{datasets_path}/Roulette.csv", sep='\t'),
}

# samples of interest: European & non-related without hom and comp_het
european_non_rel_samples = (
    ".../450k/samples/european_non_related_no_withdrawal_to_include_450k.no_hom_comp_het.txt"
)

with open(european_non_rel_samples, 'r') as f:
    european_non_rel_samples = [l.strip() for l in f.readlines()]

print (f"Number of european non-related samples without hom and comp_het: {len(european_non_rel_samples)}\n")

for dataset_name, dataset in datasets_dict.items():
    print (f"Number of samples in {dataset_name}: {datasets_dict[dataset_name].shape[0]}")
    datasets_dict[dataset_name] = dataset[dataset['eid'].astype(str).isin(european_non_rel_samples)].copy()
    print (f"Number of samples in {dataset_name} after filtration: {datasets_dict[dataset_name].shape[0]}")

Number of european non-related samples without hom and comp_het: 376608

Number of samples in Cassa: 378751
Number of samples in Cassa after filtration: 376608
Number of samples in pLI: 378751
Number of samples in pLI after filtration: 376608
Number of samples in Weghorn: 378751
Number of samples in Weghorn after filtration: 376608
Number of samples in Roulette: 378751
Number of samples in Roulette after filtration: 376608


Here are all s_het-based genetic burdens, calculated based on different gene sets and variants (PLPs or singleton LOFs):

In [3]:
for col in datasets_dict[list(datasets_dict.keys())[0]].columns:
    if col[:5] == 's_het':
        print (col)

s_het_lof_all
s_het_lof_without_AR
s_het_lof_AR
s_het_recessive_AR_without_ID
s_het_recessive_Blindness
s_het_recessive_Cardiovascular
s_het_recessive_Cilia_Kidney
s_het_recessive_Deafness
s_het_recessive_Derm
s_het_recessive_Endocrine
s_het_recessive_Hematologic
s_het_recessive_ID_total
s_het_recessive_Immune_system
s_het_recessive_Metabolic
s_het_recessive_Metabolic_ID
s_het_recessive_Neuromuscular
s_het_recessive_No_panel
s_het_recessive_Overlaps
s_het_recessive_Skeletal_Craniofacial
s_het_recessive_Tumor
s_het_recessive_all
s_het_recessive_AD
s_het_recessive_AR
s_het_recessive_ID_without_AD
s_het_recessive_sampled_AR_without_ID
s_het_recessive_sampled_ID


# Define analyses

In [4]:
covariates = ['years_of_edu', 'diagnosis_total_ICD10_cnt_log',
              'ICD_infertility', 'living_with_a_partner', 'fluid_intelligence_score']

covariate_family = ['gaussian', 'gaussian', 'binomial', 'binomial', 'gaussian']

# we looked into phenotypes of interest
phenotype_regressions = {
    's_hets': ['s_het_recessive_all', 's_het_lof_without_AR'],
    'targets': ['childlessness', 'is_blond'] + covariates,
    'families': ['binomial', 'binomial'] + covariate_family,
    'genders': ['all'], 
    'n_tests_correction': 10,
    'filter_dataset': lambda dataset: dataset
}

# we looked into these phenotypes as covariates
covariate_regressions = {
    's_hets': (['s_het_recessive_all', 's_het_lof_without_AR'] + 
               list(product(['s_het_recessive_all', 's_het_lof_without_AR'], covariates)) + 
               [('s_het_recessive_all', *covariates), ('s_het_lof_without_AR', *covariates)]),
    'targets': ['childlessness', 'is_blond'],
    'families': ['binomial', 'binomial'],
    'genders': ['all'], 
    'n_tests_correction': 12,
    'filter_dataset': lambda dataset: dataset
}

analyses = {
    'phenotype': phenotype_regressions,
    'covariate': covariate_regressions
}

# Regressions

In [5]:
for analysis, analysis_cfg in analyses.items():

    all_results = defaultdict(list)

    print (f"Running {analysis} analysis\n")

    # read analysis config
    s_hets = analysis_cfg['s_hets']
    targets = analysis_cfg['targets']
    families = analysis_cfg['families']
    genders = analysis_cfg['genders']   
    filter_dataset_func = analysis_cfg['filter_dataset']
    n_tests_correction = analysis_cfg['n_tests_correction']
     
    
    # iterate over different s-het datasets
    for dataset_key in datasets_dict:
        print ("\tProcessing", dataset_key, flush=True)

        # iterate over different s-het burdens
        for s_het in s_hets:
            if not (isinstance(s_het, list) or isinstance(s_het, tuple)) :
                s_het = [s_het]

            analysis_tag = f'{analysis} regressions on {str(s_het)}'.replace("'", '')

            print (f"\t\tRun {analysis_tag}")

            dataset = filter_dataset_func(datasets_dict[dataset_key]).copy()
            print (f"\t\t\tDataset size after filtration: {dataset.shape}")

            regression_result = run_regressions(dataset=dataset, 
                                                targets=targets, 
                                                families=families, 
                                                analysis_tag=analysis_tag, 
                                                genders=genders, 
                                                s_het_list=s_het, 
                                                tab_offset='\t\t\t', 
                                                n_tests_correction=n_tests_correction)
            
            all_results[dataset_key].append(regression_result)

            print()

    all_results = {dataset_key: pd.concat(all_results[dataset_key]) for dataset_key in all_results}

    save_table_for_paper(all_results, f"{output_path}/table_{analysis}_analysis.xlsx", 
                         keep_effects=covariates if analysis == 'phenotype' else [])

    print()

Running phenotype analysis

	Processing Cassa


		Run phenotype regressions on [s_het_recessive_all]
			Dataset size after filtration: (376608, 227)
				Processing all samples
					Processing childlessness
					Processing is_blond
					Processing years_of_edu
					Processing diagnosis_total_ICD10_cnt_log
					Processing ICD_infertility
					Processing living_with_a_partner
					Processing fluid_intelligence_score

		Run phenotype regressions on [s_het_lof_without_AR]
			Dataset size after filtration: (376608, 227)
				Processing all samples
					Processing childlessness
					Processing is_blond
					Processing years_of_edu
					Processing diagnosis_total_ICD10_cnt_log
					Processing ICD_infertility
					Processing living_with_a_partner
					Processing fluid_intelligence_score

	Processing pLI
		Run phenotype regressions on [s_het_recessive_all]
			Dataset size after filtration: (376608, 227)
				Processing all samples
					Processing childlessness
					Processing is_blond
					Processing years_of_edu
					Processing diagnosis_total

In [6]:
! ls -lah ../../../../data/tables

total 1.6M
drwxrwx--- 3 gelana bioinf  478 Aug 27 13:27 .
drwxrwx--- 5 gelana bioinf   70 Jul 17 13:02 ..
-rwxrwx--- 1 gelana bioinf 1.8K Aug  5 11:18 CR.csv
-rwxrwx--- 1 gelana bioinf  341 Aug  5 11:16 CR_dutch_estonian.csv
-rwxrwx--- 1 gelana bioinf 6.4K Aug  8 16:28 high_s_het_genes_roulette.xlsx
drwxrwx--- 5 gelana bioinf   76 Aug 19 16:38 sampling
-rwxrwx--- 1 gelana bioinf  57K Aug 28 12:05 table_covariate_analysis.xlsx
-rwxrwx--- 1 gelana bioinf  25K Aug 27 13:27 table_covariate_deprivation_analysis.xlsx
-rwxrwx--- 1 gelana bioinf  19K Aug  8 12:48 table_id_vs_rest_analysis.xlsx
-rwxrwx--- 1 gelana bioinf  78K Aug  8 12:46 table_panel_analysis.xlsx
-rwxrwx--- 1 gelana bioinf  22K Aug 28 11:57 table_phenotype_analysis.xlsx
-rwxrwx--- 1 gelana bioinf  19K Jul 18 16:21 table_selection_analysis.xlsx
-rwxrwx--- 1 gelana bioinf  39K Jul 30 13:24 table_sex_specific_analysis.xlsx
